# 07 - SymPy Foundations

## Agentic Logistics Control System

This notebook introduces SymPy symbolic mathematics for logistics modeling:
- Symbolic variables and equations
- Trajectory models
- Fuel consumption models
- Cost optimization functions

### Control Loop Role: Mathematical Foundation for PLANNING

```
OBSERVE -> REASON -> [PLAN] -> DECIDE -> ACT -> FEEDBACK -> (loop)
                       ^
                       |
              SymPy powers simulations
```

In [ ]:
# Setup
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT / "src"))

import sympy as sp
from sympy import Symbol, symbols, Eq, solve, diff, integrate, simplify, sqrt, exp
from sympy import init_printing
init_printing(use_unicode=True)

print(f"SymPy version: {sp.__version__}")

## 1. Symbolic Variables

In [ ]:
# Define core variables for logistics
t = Symbol('t', positive=True, real=True)  # time (hours)
d = Symbol('d', positive=True, real=True)  # distance (km)
v = Symbol('v', positive=True, real=True)  # velocity (km/h)
tau = Symbol('tau', positive=True, real=True)  # traffic factor
f = Symbol('f', positive=True, real=True)  # fuel rate (L/km)
W = Symbol('W', positive=True, real=True)  # weight (kg)

print("Symbolic variables defined:")
print(f"  t (time), d (distance), v (velocity)")
print(f"  tau (traffic), f (fuel rate), W (weight)")

## 2. Basic Equations

In [ ]:
# Position as function of time: x(t) = v * t
position = v * t
print("Position equation: x(t) =")
display(position)

In [ ]:
# Time to travel distance d at velocity v
travel_time = d / v
print("Travel time: t =")
display(travel_time)

In [ ]:
# Effective velocity with traffic
v_effective = v / tau
print("Effective velocity with traffic:")
display(v_effective)

In [ ]:
# Time with traffic
time_with_traffic = d / v_effective
time_with_traffic_simplified = simplify(time_with_traffic)
print("Travel time with traffic:")
display(time_with_traffic_simplified)

## 3. Fuel Consumption Models

In [ ]:
# Basic fuel consumption: F = rate * distance
fuel_basic = f * d
print("Basic fuel consumption:")
display(fuel_basic)

In [ ]:
# Fuel consumption with load weight
# F = (base_rate + weight_factor * W) * d
base_rate = Symbol('f_0', positive=True)
weight_factor = Symbol('alpha', positive=True)

fuel_with_weight = (base_rate + weight_factor * W) * d
print("Fuel consumption with load weight:")
display(fuel_with_weight)

In [ ]:
# Evaluate with specific values
fuel_example = fuel_with_weight.subs([
    (base_rate, 0.3),  # 0.3 L/km base
    (weight_factor, 0.00002),  # 0.00002 L/km per kg
    (W, 5000),  # 5000 kg load
    (d, 100)  # 100 km distance
])
print(f"Fuel for 100km with 5000kg load: {float(fuel_example):.2f} L")

## 4. Cost Functions

In [ ]:
# Cost variables
c_fuel = Symbol('c_fuel', positive=True)  # cost per liter
c_driver = Symbol('c_driver', positive=True)  # driver cost per hour
c_vehicle = Symbol('c_vehicle', positive=True)  # vehicle cost per hour

# Total cost = fuel cost + time cost
F = Symbol('F', positive=True)  # total fuel
total_cost = c_fuel * F + (c_driver + c_vehicle) * t

print("Total trip cost:")
display(total_cost)

In [ ]:
# Express cost in terms of distance and velocity
# Substitute: F = f*d and t = d/v
cost_expr = total_cost.subs([(F, f*d), (t, d/v)])
cost_simplified = simplify(cost_expr)

print("Cost as function of distance and velocity:")
display(cost_simplified)

In [ ]:
# Calculate cost for a specific trip
trip_cost = cost_simplified.subs([
    (d, 150),  # 150 km
    (v, 60),   # 60 km/h
    (f, 0.3),  # 0.3 L/km
    (c_fuel, 1.50),  # $1.50/L
    (c_driver, 25),  # $25/hour
    (c_vehicle, 10)  # $10/hour
])

print(f"Cost for 150km trip at 60km/h: ${float(trip_cost):.2f}")

## 5. Optimization: Minimizing Cost

In [ ]:
# Find optimal velocity to minimize cost
# Take derivative with respect to v and set to zero
dcost_dv = diff(cost_simplified, v)
print("Derivative of cost with respect to velocity:")
display(simplify(dcost_dv))

In [ ]:
# Solve for optimal velocity
# Since dcost/dv = -d*(c_driver + c_vehicle)/v^2, this is always negative
# Meaning cost decreases as velocity increases (ignoring fuel efficiency at high speeds)

# Let's create a more realistic model where fuel consumption increases with speed
v_opt = Symbol('v_opt', positive=True)

# Fuel rate increases quadratically from optimal speed
f_v = f * (1 + 0.01 * (v - v_opt)**2 / v_opt)
print("Speed-dependent fuel rate:")
display(f_v)

In [ ]:
# Realistic cost with speed-dependent fuel
realistic_cost = c_fuel * f_v * d + (c_driver + c_vehicle) * d / v
realistic_cost_simplified = simplify(realistic_cost)

print("Realistic cost model:")
display(realistic_cost_simplified)

## 6. Using SymbolicModels Class

In [ ]:
from src.planning.sympy_models import SymbolicModels

models = SymbolicModels()
print("SymbolicModels loaded!")

In [ ]:
# Test trajectory calculation
segments = [
    (50, 80, 1.0),   # 50km at 80km/h, no traffic
    (30, 60, 1.5),   # 30km at 60km/h, moderate traffic
    (20, 60, 2.0),   # 20km at 60km/h, heavy traffic
]

total_time, total_dist = models.trajectory_with_traffic(segments)
print(f"Route with multiple segments:")
print(f"  Total distance: {total_dist} km")
print(f"  Total time: {float(total_time):.2f} hours ({float(total_time)*60:.0f} minutes)")

In [ ]:
# Compare routes
routes = [
    {'name': 'Highway I-95', 'distance_km': 100, 'traffic_factor': 1.8, 'base_speed_kmh': 80},
    {'name': 'Route 9A', 'distance_km': 110, 'traffic_factor': 1.1, 'base_speed_kmh': 65},
    {'name': 'Local Roads', 'distance_km': 85, 'traffic_factor': 1.2, 'base_speed_kmh': 50},
]

comparison = models.route_comparison(routes)

print("Route Comparison:")
print("-" * 60)
for name, metrics in comparison.items():
    print(f"\n{name}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value}")

In [ ]:
# Optimize for deadline
result = models.minimize_cost_with_constraint(
    max_time=2.0,  # 2 hours available
    distance=100,  # 100 km to go
    min_speed=30,
    max_speed=100
)

print("Optimization Result:")
for key, value in result.items():
    print(f"  {key}: {value}")

## 7. Sensitivity Analysis

In [ ]:
# Analyze sensitivity of time to velocity changes
time_expr = models.d / models.v

# Substitute d=100
time_100km = time_expr.subs(models.d, 100)

sensitivity = models.sensitivity_analysis(
    expr=time_100km,
    variable='v',
    base_value=60,
    range_percent=20
)

print("Sensitivity of travel time to velocity:")
print(f"  At 48 km/h: {sensitivity['low_result']:.2f} hours")
print(f"  At 60 km/h: {sensitivity['base_result']:.2f} hours")
print(f"  At 72 km/h: {sensitivity['high_result']:.2f} hours")
print(f"  Sensitivity: {sensitivity['sensitivity']:.4f} hours per km/h")

## 8. Visualization

In [ ]:
# Plot cost vs velocity
from sympy import plotting

# Create cost expression with fixed values except velocity
v_var = Symbol('v', positive=True)
cost_plot = 0.3 * 100 * 1.5 + (25 + 10) * 100 / v_var  # Simplified cost for 100km

print("Cost vs Velocity plot (for 100km trip):")
p = plotting.plot(cost_plot, (v_var, 30, 100), 
                  xlabel='Velocity (km/h)', 
                  ylabel='Cost ($)',
                  title='Trip Cost vs Speed',
                  show=False)
p.show()

## Integration Test

In [ ]:
print("=" * 50)
print("SYMPY FOUNDATIONS TEST")
print("=" * 50)

tests_passed = 0
tests_total = 0

# Test 1: Basic symbolic computation
tests_total += 1
try:
    expr = Symbol('x') + Symbol('y')
    result = expr.subs([('x', 2), ('y', 3)])
    assert result == 5
    print("✓ Test 1: Basic symbolic computation")
    tests_passed += 1
except Exception as e:
    print(f"✗ Test 1: {e}")

# Test 2: SymbolicModels initialization
tests_total += 1
try:
    models = SymbolicModels()
    assert models.t is not None
    print("✓ Test 2: SymbolicModels initialization")
    tests_passed += 1
except Exception as e:
    print(f"✗ Test 2: {e}")

# Test 3: Time calculation
tests_total += 1
try:
    time_result = models.time_to_destination(100, 50)
    assert float(time_result) == 2.0
    print("✓ Test 3: Time calculation")
    tests_passed += 1
except Exception as e:
    print(f"✗ Test 3: {e}")

# Test 4: Fuel consumption
tests_total += 1
try:
    fuel = models.fuel_consumption_basic(100, 0.3)
    assert float(fuel) == 30.0
    print("✓ Test 4: Fuel consumption")
    tests_passed += 1
except Exception as e:
    print(f"✗ Test 4: {e}")

# Test 5: Route comparison
tests_total += 1
try:
    routes = [
        {'name': 'A', 'distance_km': 100, 'traffic_factor': 1.0},
        {'name': 'B', 'distance_km': 110, 'traffic_factor': 1.1},
    ]
    comparison = models.route_comparison(routes)
    assert 'A' in comparison
    assert 'B' in comparison
    print("✓ Test 5: Route comparison")
    tests_passed += 1
except Exception as e:
    print(f"✗ Test 5: {e}")

print("=" * 50)
print(f"Tests passed: {tests_passed}/{tests_total}")
if tests_passed == tests_total:
    print("✓ All tests passed!")

## Next Steps

1. Proceed to `08_simulation_engine.ipynb` for full simulation capabilities
2. Then `09_scenario_generator.ipynb` for scenario planning